In [128]:
# bf.get_boot_attrs(connection)

def bytes_to_hex_string(byte_data, addSpaces = True):
    hex_bytes = [f"{byte:02x}" for byte in bytes(byte_data)]
    if addSpaces:
        return ' '.join(hex_bytes, )
    else:
        return ''.join(hex_bytes, )



In [66]:
from _types import *

# la première commande est : # bf.get_boot_attrs(connection)
# qui fait : 
    # mem_range_response = _send_and_receive(
    #     connection,
    #     Command(CommandCode.GET_MEMORY_ADDRESS_RANGE),
    # )

# Checksum(42).__bytes__()

# len(bytes(Command(CommandCode.GET_MEMORY_ADDRESS_RANGE)))

# # ok, la commande pour demander fait 11 octets, et commence juste par "0b"
# # ensuite, déparser la réponse
v = Version(ResponseCode.SUCCESS)
v.version = 42
v.max_packet_length = 43
v.device_id = 45
v.erase_size = 48
v.write_size = 34

bytes_to_hex_string(v.__bytes__())


'01 00 00 00 00 00 00 00 00 00 00 2a 00 2b 00 00 00 2d 00 00 00 30 00 22 00 00 00 00 00 00 00 00 00 00 00 00 00'

In [113]:
import importlib
import _util
importlib.reload(_util)

bootattrs = BootAttrs(
        version=258,
        max_packet_length=256,
        device_id=13398,
        erase_size=2048,
        write_size=8,
        memory_range=(6144, 174080),
        has_checksum=True,
    )
total_bytes, chunks = _util.chunked("../mcbootflash/tests/testcases/flash/test.hex", bootattrs)

# chunks_array = []
# for chunk in chunks:
#     print("CHUNK : ")
#     print(chunk.address)
#     print(bytes_to_hex_string(chunk.data))
#     print("        ")



number of segments 2
first segment  Segment(address=0, data=bytearray(b'\xe0\x1a\x04\x00\x00\x00\x00\x00\x04\x1a\x00\x00\x08\x1a\x00\x00\x0c\x1a\x00\x00\x10\x1a\x00\x00\x14\x1a\x00\x00\x18\x1a\x00\x00\x1c\x1a\x00\x00\x00\x1a\x00\x00 \x1a\x00\x00$\x1a\x00\x00(\x1a\x00\x00,\x1a\x00\x000\x1a\x00\x004\x1a\x00\x008\x1a\x00\x00<\x1a\x00\x00@\x1a\x00\x00D\x1a\x00\x00H\x1a\x00\x00L\x1a\x00\x00P\x1a\x00\x00T\x1a\x00\x00X\x1a\x00\x00\x00\x1a\x00\x00\\\x1a\x00\x00`\x1a\x00\x00d\x1a\x00\x00h\x1a\x00\x00l\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00p\x1a\x00\x00t\x1a\x00\x00x\x1a\x00\x00|\x1a\x00\x00\x80\x1a\x00\x00\x84\x1a\x00\x00\x88\x1a\x00\x00\x8c\x1a\x00\x00\x90\x1a\x00\x00\x94\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x98\x1a\x00\x00\x9c\x1a\x00\x00\xa0\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\x00\x1a\x00\x00\xa4\x1a\x00\x00\xa8\x1a\x00\x00\x00\x1a\x00\x00\x00\x1

In [143]:
# unpack_ihex test
def crc_ihex(hexstr):
    """Calculate the CRC for given Intel HEX hexstring.

    """

    crc = sum(binascii.unhexlify(hexstr))

    print("sum of crc : ", crc)
    crc &= 0xff
    print("after &= : ", crc)
    crc = ((~crc + 1) & 0xff)
    print("final crc : ", crc)

    return crc

def unpack_ihex(record):
    """Unpack given Intel HEX record into variables.

    """

    # Minimum :SSAAAATTCC, where SS is size, AAAA is address, TT is
    # type and CC is crc.
    if len(record) < 11:
        raise Error(f"record '{record}' too short")

    if record[0] != ':':
        raise Error(f"record '{record}' not starting with a ':'")

    value = bytearray.fromhex(record[1:])
    size = value[0]

    if size != len(value) - 5:
        raise Error(f"record '{record}' has wrong size")

    address = int.from_bytes(value[1:3], byteorder='big')
    type_ = value[3]
    data = value[4:-1]
    actual_crc = value[-1]

    crcBytes = record[1:-2]
    expected_crc = crc_ihex(crcBytes)
    print("crcBytes")
    print(crcBytes)

    print("expected_crc ", expected_crc)

    if actual_crc != expected_crc:
        raise Error(
            f"expected crc '{expected_crc:02X}' in record {record}, but got "
            f"'{actual_crc:02X}'")

    return (type_, address, size, data)


(type_, address, size, data) = unpack_ihex(":10000000E01A040000000000041A0000081A0000B2")
print("type_", type_)
print("address", address)
print("size", size)
print("data", bytes_to_hex_string(data))

# 10000000e01a040000000000041a0000081a0000

# from C++ : 

# 10000000e01a040000000000041a0000081a0000

sum of crc :  334
after &= :  78
final crc :  178
crcBytes
10000000E01A040000000000041A0000081A0000
expected_crc  178
type_ 0
address 0
size 16
data e0 1a 04 00 00 00 00 00 04 1a 00 00 08 1a 00 00


In [150]:
expected_crc = crc_ihex("4338592D")
expected_crc

sum of crc :  257
after &= :  1
final crc :  255


255

In [ ]:
# Connect to a device in bootloader mode.
connection = serial.Serial(port=<PORT>, baudrate=<BAUDRATE>, timeout=<TIMEOUT>)
# Query its attributes.
bootattrs = bf.get_boot_attrs(connection)
# Load the firmware image and split it into chunks.
total_bytes, chunks = bf.chunked(hexfile=<HEXFILE_PATH_STRING>, bootattrs)
# Erase the device's program memory area.
bf.erase_flash(connection, bootattrs.memory_range, bootattrs.erase_size)

# Write the firmware chunks to the bootloader in a loop.
for chunk in chunks:
    bf.write_flash(connection, chunk)

    # Optionally, check that the write is OK by checksumming.
    if bootattrs.has_checksum:
        bf.checksum(connection, chunk)

    # At this point, you may want to give an indication of the flashing progress,
    # like updating a progress bar.

# Verify that the new application is detected.
bf.self_verify(connection)


In [98]:
# test du crc ! 
import binascii
def crc_srec(hexstr):
    """Calculate the CRC for given Motorola S-Record hexstring.

    """

    crc = sum(binascii.unhexlify(hexstr))
    crc &= 0xff
    crc ^= 0xff

    return crc

#    std::vector<uint8_t> bytes{67, 56, 89, 45};
print(crc_srec("4338592d") == 254)


crc_srec("588405bc7be1f154bbab51427e254050425dcf55") == 210

True


True

les étapes du code : 

get_boot_attrs : 

# demander VERSION
    read_version_response = _send_and_receive(
        connection,
        Command(CommandCode.READ_VERSION),
    )

# demander MEMORY_RANGE
    mem_range_response = _send_and_receive(
        connection,
        Command(CommandCode.GET_MEMORY_ADDRESS_RANGE),
    )

    assert isinstance(mem_range_response, MemoryRange)

    # program_end + 2 explanation:
    # +1 because the upper bound reported by the bootloader is inclusive, but we
    # want to use it as a Python range, which is half-open.
    # +1 because the final byte of the final 24-bit instruction is not included in
    # the range reported by the bootloader, but it is still writable.
    return mem_range_response.program_start, mem_range_response.program_end + 2


# demander CHECKSUM (peut planter)
    try:
        _get_remote_checksum(connection, memory_range[0], write_size)
        has_checksum = True
    except UnsupportedCommand:
        _logger.warning("Bootloader does not support checksumming")
        has_checksum = False

    checksum_response = _send_and_receive(
        connection,
        Command(
            command=CommandCode.CALC_CHECKSUM,
            data_length=length,
            address=address,
        ),
    )
    assert isinstance(checksum_response, Checksum)

# on a tous les boot_attrs : 
    return BootAttrs(
        version,
        max_packet_length,
        device_id,
        erase_size,
        write_size,
        memory_range,
        has_checksum,
    )


# ensuite, on chunke tout : 

total_bytes, chunks = bf.chunked(hexfile=<HEXFILE_PATH_STRING>, bootattrs)


# on erase la flash : 
    _send_and_receive(
        connection,
        command=Command(
            command=CommandCode.ERASE_FLASH,
            data_length=(end - start) // erase_size,
            unlock_sequence=_FLASH_UNLOCK_KEY,
            address=start,
        ),
    )


# pour tous les chunks, on fait : 

    _send_and_receive(
        connection,
        Command(
            command=CommandCode.WRITE_FLASH,
            data_length=len(chunk.data),
            unlock_sequence=_FLASH_UNLOCK_KEY,
            address=chunk.address,
        ),
        chunk.data,
    )






In [ ]:
# ok, il faut tester l'opération de chunkage maintenant, le reste y'a pas grand chose : 


In [49]:
[x for x in "Hello world ! "].reverse()